In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

ds = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=Lambda(lambda y: torch.zeros(
        10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [12]:
print(logits)
logits.shape

tensor([[-0.0469, -0.0006,  0.0452, -0.0687, -0.0005,  0.0196,  0.0815, -0.0211,
          0.0083, -0.0447]], device='cuda:0', grad_fn=<AddmmBackward0>)


torch.Size([1, 10])

In [13]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [14]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [15]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [16]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.4651, -0.5765,  0.1739,  0.1035, -0.2400,  0.1000,  0.9010,  0.6794,
         -0.2042, -0.7265, -0.2060,  0.0538, -0.0598,  0.1056,  0.3359, -0.2771,
          0.0412,  0.0439, -0.0830, -0.3992],
        [ 0.1491, -0.2677,  0.2656,  0.5400, -0.1535,  0.1897,  0.8267,  0.1153,
         -0.1734, -0.8901,  0.2443, -0.1968, -0.4552,  0.2998,  0.5548, -0.2469,
         -0.1557,  0.1648,  0.1565, -0.2231],
        [ 0.2482, -0.6943, -0.2859,  0.2422, -0.2232,  0.3032,  0.6528,  0.2251,
         -0.7097, -0.8907,  0.2375, -0.2824, -0.0167,  0.0128,  0.3214, -0.2821,
         -0.4373,  0.1931,  0.2673, -0.5096]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4651, 0.0000, 0.1739, 0.1035, 0.0000, 0.1000, 0.9010, 0.6794, 0.0000,
         0.0000, 0.0000, 0.0538, 0.0000, 0.1056, 0.3359, 0.0000, 0.0412, 0.0439,
         0.0000, 0.0000],
        [0.1491, 0.0000, 0.2656, 0.5400, 0.0000, 0.1897, 0.8267, 0.1153, 0.0000,
         0.0000, 0.2443, 0.0000, 0.0000, 0.2998, 0.55

In [17]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0224, -0.0076, -0.0255,  ..., -0.0289,  0.0260, -0.0134],
        [ 0.0040, -0.0113, -0.0214,  ..., -0.0087,  0.0071, -0.0076]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0267,  0.0127], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0189, -0.0016, -0.0062,  ...,  0.0053, -0.0188,  0.0076],
        [-0.0293,  0.0265, -0.0036,  ..., -0.0299, -0.0396,  0.0217]],
       device='cuda:0', grad_fn=<Slice